In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
import matplotlib.pyplot as plt
import torchtext
import spacy
import tiktoken
import pandas as pd

from gensim.corpora.dictionary import Dictionary

from tqdm.notebook import tqdm

In [2]:
import os
en = spacy.load('en_core_web_sm')
de = spacy.load('de_core_news_sm')
#spacy.load('en_core_web_sm')
#spacy.load('de_core_news_sm')

#dataset https://nlp.stanford.edu/projects/nmt/

rough work

In [3]:
test = "hello my name is hello name is my"
test2 = "one two three four"

doc_ = en(test)

for token in doc_:
    print(token.text.lower())
sentence_ = [[token.text for token in doc_]]
#sentence = [sentence + ['one','two']]

# sentence

sentence2 = [['one', 'two', 'hello']]

hello
my
name
is
hello
name
is
my


In [4]:
vocab = Dictionary()

vocab.add_documents(sentence_)

In [5]:
df = pd.DataFrame()

In [6]:
vocab_en = Dictionary()

# vocab_en.add_documents([["<START>"]])
# vocab_en.add_documents([["<END>"]])
vocab_en.add_documents([["<PAD>"]])

vocab_ger = Dictionary()

vocab_ger.add_documents([["<START>"]])
vocab_ger.add_documents([["<END>"]])
vocab_ger.add_documents([["<PAD>"]])

max in german text is 119
max in english text is 109

pad both to 128

15 minutes for 100000 sentences, both german and english

328514

In [10]:
en_max = 0 
with open(os.getcwd()+'\\data\\train_en.txt', 'r', encoding='utf8') as f:
    idx_en = []
    sentences_en = []
    for i in tqdm(range(400000)):
        line = f.readline()
        line = line.replace("\n", "")
        len_pad = 0
        doc = en(line)
        sentence = [[token.text.lower() for token in doc]]
        if len(sentence[0]) > en_max:
            en_max = len(sentence[0])
            print(en_max)
        vocab_en.add_documents(sentence)
        sentence = ["<PAD>"] + sentence[0] + ["<PAD>"]

        if len(sentence) <= 256:
            len_pad = 256 - len(sentence)
            sentence = sentence + len_pad*"<PAD> ".split()
            assert len(sentence) == 256, print(len(sentence))
            idx_en.append(i)
            sentences_en.append(sentence)

print(en_max)    
print(len(sentences_en))
print(len(vocab_en.token2id))

  0%|          | 0/400000 [00:00<?, ?it/s]

32
34
47
49
54
57
58
67
87
91
92
97
98
100
105
117
130
153
169
182
244
245
249
342
342
399999
176008


In [ ]:
de_max = 0 
with open(os.getcwd()+'\\data\\de_edit.txt', 'r', encoding='utf8') as f:
    sentences_de = []
    for i in tqdm(range(400000)):
        line = f.readline()
        line = line.replace("\n", "")
        len_pad = 0
        doc = de(line)
        sentence = [[token.text.lower() for token in doc]]
        if len(sentence[0]) > de_max:
            de_max = len(sentence[0])
            print(de_max)
        vocab_en.add_documents(sentence)
        sentence = ["<START>"] + sentence[0] + ["<END>"]
        if len(sentence) < 256:
            len_pad = 256 - len(sentence)
            sentence = sentence + len_pad*"<PAD> ".split()
            assert len(sentence) == 256, print(len(sentence))
            idx_en.append(i)
            sentences_de.append(sentence)

print(de_max)  
print(len(sentences_de))
print(len(vocab_en.token2id))

In [15]:
df['idx_en'] = idx_en
df['inputs'] = sentences_en
#df['targets'] = sentences_de

In [16]:
df.head()

,inputs,idx_en
0,"[<PAD>, iron, cement, is, a, ready, for, use, ...",0
1,"[<PAD>, iron, cement, protects, the, ingot, ag...",1
2,"[<PAD>, a, fire, restant, repair, cement, for,...",2
3,"[<PAD>, construction, and, repair, of, highway...",3
4,"[<PAD>, an, announcement, must, be, commercial...",4


In [17]:
len(df)

399999

In [18]:
df.to_csv(os.getcwd()+"\\data\\data.csv", index = False)

In [19]:
df['inputs'].values[0]

['<PAD>',
 'iron',
 'cement',
 'is',
 'a',
 'ready',
 'for',
 'use',
 'paste',
 'which',
 'is',
 'laid',
 'as',
 'a',
 'fillet',
 'by',
 'putty',
 'knife',
 'or',
 'finger',
 'in',
 'the',
 'mould',
 'edges',
 '(',
 'corners',
 ')',
 'of',
 'the',
 'steel',
 'ingot',
 'mould',
 '.',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD

In [20]:
vocab_en.doc2idx(df['inputs'].values[0])

[0,
 15,
 7,
 16,
 4,
 24,
 12,
 27,
 22,
 28,
 16,
 18,
 5,
 4,
 10,
 6,
 23,
 17,
 21,
 11,
 13,
 26,
 19,
 9,
 1,
 8,
 2,
 20,
 26,
 25,
 14,
 19,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0